This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [3]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

FuM91c6PyEVWBvnufuKD


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [77]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

r = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2023-05-14&end_date=2023-15-14api_key=" + API_KEY)
# r.text

In [6]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
res = json.loads(r.text) 
# print(json.dumps(res, indent=4, sort_keys=True))

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [70]:
# Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2020 (keep in mind that the date format is YYYY-MM-DD).
r = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2020-01-01&end_date=2021-01-01&api_key=" + API_KEY)
# r.text

In [64]:
# Convert the returned JSON object into a Python dictionary.
res = json.loads(r.text) 
# print(json.dumps(res, indent=4, sort_keys=True))

In [65]:
columns = res['dataset']['column_names']
print(columns)

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [67]:
data = res['dataset']['data']
# print(json.dumps(data, indent=4, sort_keys=True))

In [68]:
# Calculate what the highest and lowest opening prices were for the stock in this period.
min_open_price = data[0][1]
max_open_price = data[0][1]
for d in data[1:]:
    if d[1] is not None:
        if d[1] > max_open_price:
            max_open_price = d[1]
        elif d[1] < min_open_price:
            min_open_price = d[1]
# print(max_open_price)
# print(min_open_price)
print("Lowest Opening Prices: $%.2f. \nHighest Opening Prices: $%.2f." % (min_open_price, max_open_price))

Lowest Opening Prices: $69.15. 
Highest Opening Prices: $122.10.


In [69]:
# What was the largest change in any one day (based on High and Low price)?
largest_change_in_price = data[0][2] -  data[0][3]
largest_change_in_price_date = data[0][0]
for d in data[1:]:
    if d[2] is not None and d[3] is not None:
        change = d[2] -  d[3]
        if change > largest_change_in_price:
            largest_change_in_price = change
            largest_change_in_price_date = d[0]
    
# print(largest_change_in_price)
# print(largest_change_in_price_date)

print("The largest change in any one day was $%.2f on %s." % (largest_change_in_price, largest_change_in_price_date))

The largest change in any one day was $14.10 on 2020-02-10.


In [71]:
# What was the largest change between any two days (based on Closing Price)?
largest_change_in_2_day_price = data[0][4] -  data[1][4]
largest_change_in_2_day_price_date_1 = data[0][0]
largest_change_in_2_day_price_date_2 = data[1][0]
for i, d in enumerate(data[1:-1]):
#     print(data[i][0], '\t', data[i+1][0], '\t', data[i+1][4], '\t',  data[i][4], '\t', data[i+1][4] - data[i][4])
    change = data[i+1][4] - data[i][4]
    if change > largest_change_in_2_day_price:
        largest_change_in_2_day_price = change
        largest_change_in_2_day_price_date_1 = data[i+1][0]
        largest_change_in_2_day_price_date_2 = data[i][0]
    
print(largest_change_in_2_day_price)
print(largest_change_in_2_day_price_date_1)
print(largest_change_in_2_day_price_date_2)

print("The largest change between two days is $%.2f from %s to %s." % (largest_change_in_2_day_price,largest_change_in_2_day_price_date_1,largest_change_in_2_day_price_date_2))

7.849999999999994
2020-03-11
2020-03-12
The largest change between two days is $7.85 from 2020-03-11 to 2020-03-12.


In [72]:
# What was the average daily trading volume during this year?
count_volumes = data[0][6]
open_days = 1
for d in data[1:]:
    count_volumes = count_volumes + d[6]
    open_days = open_days + 1
average_volumes = count_volumes / open_days
print(average_volumes)

print("The average daily trading volume is $%.2f " % (average_volumes))

126294.17446808511
The average daily trading volume is $126294.17 


In [73]:
# 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
half_1 = int((open_days/2)-1)
half_2 = int(open_days/2)
median_volumns = (data[half_1][6] + data[half_2][6] )/2
print(median_volumns)

print("The median trading volume during this year was $%.2f " % (median_volumns))

145878.5
The median trading volume during this year was $145878.50 


In [74]:
print("Lowest Opening Prices: $%.2f. \nHighest Opening Prices: $%.2f." % (min_open_price, max_open_price))
print("The largest change in any one day was $%.2f on %s." % (largest_change_in_price, largest_change_in_price_date))
print("The largest change between two days is $%.2f from %s to %s." % (largest_change_in_2_day_price, largest_change_in_2_day_price_date_1, largest_change_in_2_day_price_date_2))
print("The average daily trading volume is $%.2f " % (average_volumes))
print("The median trading volume during this year was $%.2f " % (median_volumns))

Lowest Opening Prices: $69.15. 
Highest Opening Prices: $122.10.
The largest change in any one day was $14.10 on 2020-02-10.
The largest change between two days is $7.85 from 2020-03-11 to 2020-03-12.
The average daily trading volume is $126294.17 
The median trading volume during this year was $145878.50 
